In [1]:
# Necessary modules
import pandas as pd
import numpy  as np

from keras.models     import Sequential
from keras.layers     import Dense, Dropout, Activation, Flatten
from keras.layers     import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [2]:
# Get the data
datas = []
foldsToRanges = []
start, end = 0, 0
for fold in range(5):
    path = "../data/face_image_project/fold_%d_data.txt" % (fold)
    data_fold = pd.read_csv(filepath_or_buffer=path, sep="\t")
    data_fold = data_fold.sample(frac=1).reset_index(drop=True)    
    datas.append(data_fold)
    
    start = end
    end = start + len(data_fold)
    foldsToRanges.append(slice(start, end))

In [55]:
foldsToRanges

[slice(0, 4484, None),
 slice(4484, 8214, None),
 slice(8214, 12108, None),
 slice(12108, 15554, None),
 slice(15554, 19370, None)]

In [3]:
data = pd.concat(datas, ignore_index=True)
data.count()
(data.age == 'None').sum()

748

In [4]:
data = data[["user_id","original_image","face_id","gender", "age"]]
data.head()

,user_id,original_image,face_id,gender,age
0,20254529@N04,9405946772_fbe0c4d242_o.jpg,14,f,"(4, 6)"
1,29671106@N00,8940895292_ae98e927a8_o.jpg,186,u,"(0, 2)"
2,30601258@N03,9350727922_14105fcae9_o.jpg,7,f,"(25, 32)"
3,68825596@N05,11594348474_7995fc5d07_o.jpg,296,f,"(4, 6)"
4,7648211@N03,10198608974_6e4b86bbc7_o.jpg,456,u,"(0, 2)"


In [5]:
data = data[data.gender != "u"]
data.gender.value_counts()

f    9372
m    8120
Name: gender, dtype: int64

In [6]:
data.loc[data.gender == "f", "gender"] = 0
data.loc[data.gender == "m", "gender"] = 1

In [7]:
path_template = "../data/face_image_project/aligned/%s/landmark_aligned_face.%d.%s"
data["file_path"] = data[["user_id","face_id","original_image"]].apply(lambda x:  
                                                                   path_template % (x[0],x[1],x[2]),
                                                                   axis=1)

In [8]:
data.age.value_counts()

(25, 32)     5002
(38, 43)     2293
(4, 6)       2139
(8, 12)      2122
(15, 20)     1640
(0, 2)       1397
(60, 100)     871
(48, 53)      830
None          748
35            293
13            168
22            149
34            105
23             96
45             88
(27, 32)       77
55             76
36             56
(38, 42)       46
57             24
3              18
29             11
(38, 48)        6
58              5
2               3
32              3
56              2
42              1
46              1
(8, 23)         1
Name: age, dtype: int64

In [9]:
data["age"] = data["age"].replace("13","(8, 13)")
data["age"] = data["age"].replace("(8, 12)","(8, 13)")
data["age"] = data["age"].replace("42","(38, 42)")
data["age"] = data["age"].replace("2","(0, 2)")
data["age"] = data["age"].replace("29","(25, 32)")

data["age"] = data["age"].replace("35","(25, 32)")
data["age"] = data["age"].replace("22","(15, 20)")
data["age"] = data["age"].replace("34","(25, 32)")
data["age"] = data["age"].replace("23","(25, 32)")
data["age"] = data["age"].replace("45","(48, 53)")
data["age"] = data["age"].replace("55","(48, 53)")
data["age"] = data["age"].replace("36","(38, 43)")
data["age"] = data["age"].replace("3","(0, 2)")


data["age"] = data["age"].replace("57","(60, 100)")
data["age"] = data["age"].replace("58","(60, 100)")
data["age"] = data["age"].replace("56","(60, 100)")
data["age"] = data["age"].replace("46","(48, 53)")

In [10]:
data.age.value_counts()

(25, 32)     5507
(38, 43)     2349
(8, 13)      2290
(4, 6)       2139
(15, 20)     1789
(0, 2)       1418
(48, 53)      995
(60, 100)     902
None          748
(27, 32)       77
(38, 42)       47
(38, 48)        6
32              3
(8, 23)         1
Name: age, dtype: int64

### Lets make no ranges overlap

In [11]:
data["age"] = data["age"].replace("(38, 42)","(38, 48)")
data["age"] = data["age"].replace("(8, 13)","(8, 23)")
data["age"] = data["age"].replace("32","(25, 32)")
data["age"] = data["age"].replace("(27, 32)","(25, 32)")
data["age"] = data["age"].replace("(38, 43)","(38, 48)")

In [12]:
data.age.value_counts()

(25, 32)     5587
(38, 48)     2402
(8, 23)      2291
(4, 6)       2139
(15, 20)     1789
(0, 2)       1418
(48, 53)      995
(60, 100)     902
None          748
Name: age, dtype: int64

In [13]:
# Copy original before we drop stuff
data_orig = data

In [14]:
## Drop the None data
data = data[data.age != 'None']

In [18]:
# Building a vainilla CNN
vainilla_cnn = Sequential()
vainilla_cnn.add(Convolution2D(32, 3, 3, input_shape=(227,227,3)))
vainilla_cnn.add(Activation('relu'))
vainilla_cnn.add(MaxPooling2D(pool_size=(2, 2)))
vainilla_cnn.add(Convolution2D(64, 3, 3))
vainilla_cnn.add(Activation('relu'))
vainilla_cnn.add(Convolution2D(64, 3, 3))
vainilla_cnn.add(Activation('relu'))
vainilla_cnn.add(MaxPooling2D(pool_size=(2, 2)))
vainilla_cnn.add(Flatten())
vainilla_cnn.add(Dense(output_dim=200, input_dim=500))
vainilla_cnn.add(BatchNormalization())
vainilla_cnn.add(Activation("relu"))
vainilla_cnn.add(Dense(output_dim=100, input_dim=200))
vainilla_cnn.add(BatchNormalization())
vainilla_cnn.add(Activation("relu"))
vainilla_cnn.add(Dense(output_dim=8))
vainilla_cnn.add(Activation("softmax"))

In [16]:
vainilla_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
data = data[["file_path","gender", "age"]]
data = pd.get_dummies(data, columns=["age"])

In [18]:
data.columns

Index([u'file_path', u'gender', u'age_(0, 2)', u'age_(15, 20)',
       u'age_(25, 32)', u'age_(38, 48)', u'age_(4, 6)', u'age_(48, 53)',
       u'age_(60, 100)', u'age_(8, 23)'],
      dtype='object')

In [19]:
age_cols = [u'age_(0, 2)', u'age_(15, 20)',
       u'age_(25, 32)', u'age_(38, 48)', u'age_(4, 6)', u'age_(48, 53)',
       u'age_(60, 100)', u'age_(8, 23)']
len(age_cols)

8

In [21]:
import scipy.misc

i_width  = 227
i_height = 227

def train_generator(data):
    while True:
        start, end = 0, 32
        while end < len(data):
            data = data.sample(frac=1).reset_index(drop=True)
            sample  = data[start:end]

            X = pd.DataFrame(sample["file_path"].apply(lambda x:  img_to_array( scipy.misc.imresize(load_img(x), (i_height, i_width) ) ) ))           
            X = X["file_path"].apply(lambda x: x.reshape((1,)+ x.shape))
            X = np.vstack(X)

            Y = sample[age_cols].as_matrix()
            
            yield (X, Y)
            start += 32
            end += 32

In [22]:
a,b = train_generator(data).next()
print a.shape
print b.shape

(32, 227, 227, 3)
(32, 8)


In [23]:
data_orig = data_orig[["file_path","gender", "age"]]
data_orig = pd.get_dummies(data_orig, columns=["age"])

fold0to3 = pd.concat([data_orig[foldsToRanges[0]], data_orig[foldsToRanges[1]], data_orig[foldsToRanges[2]], 
                     data_orig[foldsToRanges[3]]])
fold0to3 = fold0to3[fold0to3.age_None != 1]
fold0to3.head()

,file_path,gender,"age_(0, 2)","age_(15, 20)","age_(25, 32)","age_(38, 48)","age_(4, 6)","age_(48, 53)","age_(60, 100)","age_(8, 23)",age_None
0,../data/face_image_project/aligned/16166376@N0...,0,0,0,0,0,1,0,0,0,0
1,../data/face_image_project/aligned/37303189@N0...,1,1,0,0,0,0,0,0,0,0
2,../data/face_image_project/aligned/20254529@N0...,0,0,0,0,0,1,0,0,0,0
3,../data/face_image_project/aligned/10406201@N0...,1,0,0,0,1,0,0,0,0,0
4,../data/face_image_project/aligned/37303189@N0...,1,1,0,0,0,0,0,0,0,0


In [ ]:
history = vainilla_cnn.fit_generator(train_generator(fold0to3), samples_per_epoch=64, nb_epoch=2000)
vainilla_cnn.save_weights('vainilla_cnn_age_1_trainfold0to3.h5')

Epoch 1/2000
64/64 [==============================] - 6s - loss: 2.3749 - acc: 0.2188     
Epoch 2/2000
64/64 [==============================] - 1s - loss: 2.2099 - acc: 0.1562     
Epoch 3/2000
64/64 [==============================] - 1s - loss: 2.2887 - acc: 0.1406     
Epoch 4/2000
64/64 [==============================] - 1s - loss: 2.1873 - acc: 0.1875     
Epoch 5/2000
64/64 [==============================] - 1s - loss: 2.1578 - acc: 0.2031     
Epoch 6/2000
64/64 [==============================] - 1s - loss: 2.0443 - acc: 0.2188     

In [22]:
### UNCOMMENT ME NEXT TIME U RUN. DONT TRAIN THE DAMN NET AGAIN
vainilla_cnn.load_weights('vainilla_cnn_age_1_trainfold0to3.h5')

### Test

In [51]:
fold4 = data_orig[foldsToRanges[4]]
fold4 = fold4[fold4.age_None != 1]
fold4.head()

,file_path,gender,"age_(0, 2)","age_(15, 20)","age_(25, 32)","age_(38, 48)","age_(4, 6)","age_(48, 53)","age_(60, 100)","age_(8, 23)",age_None
16444,../data/face_image_project/aligned/10069023@N0...,0,0,0,1,0,0,0,0,0,0
16445,../data/face_image_project/aligned/86629393@N0...,1,0,0,0,0,1,0,0,0,0
16446,../data/face_image_project/aligned/10255165@N0...,1,0,0,0,0,0,0,0,1,0
16447,../data/face_image_project/aligned/10069023@N0...,0,0,0,1,0,0,0,0,0,0
16448,../data/face_image_project/aligned/10069023@N0...,0,0,0,0,1,0,0,0,0,0


In [52]:
len(fold4)

2666

In [21]:
vainilla_cnn.evaluate_generator(train_generator(fold4), 32*400)

[2.9907440868020059, 0.39695312500000002]